In [ ]:
#what is in the shared Space?

In [1]:
%%bash
ls /shared_space/ACV/summary_files

monthly_max.nc
monthly_mean.nc


In [20]:
%%bash
ls /shared_space/ACV/extracted_raw/mag_dir

mag_dir197910.nc
mag_dir197911.nc
mag_dir197912.nc
mag_dir198001.nc
mag_dir198002.nc
mag_dir198003.nc
mag_dir198004.nc
mag_dir198005.nc
mag_dir198006.nc
mag_dir198007.nc
mag_dir198008.nc
mag_dir198009.nc
mag_dir198010.nc
mag_dir198011.nc
mag_dir198012.nc
mag_dir198101.nc
mag_dir198102.nc
mag_dir198103.nc
mag_dir198104.nc
mag_dir198105.nc
mag_dir198106.nc
mag_dir198107.nc
mag_dir198108.nc
mag_dir198109.nc
mag_dir198110.nc
mag_dir198111.nc
mag_dir198112.nc
mag_dir198201.nc
mag_dir198202.nc
mag_dir198203.nc
mag_dir198204.nc
mag_dir198205.nc
mag_dir198206.nc
mag_dir198207.nc
mag_dir198208.nc
mag_dir198209.nc
mag_dir198210.nc
mag_dir198211.nc
mag_dir198212.nc
mag_dir198301.nc
mag_dir198302.nc
mag_dir198303.nc
mag_dir198304.nc
mag_dir198305.nc
mag_dir198306.nc
mag_dir198307.nc
mag_dir198308.nc
mag_dir198309.nc
mag_dir198310.nc
mag_dir198311.nc
mag_dir198312.nc
mag_dir198401.nc
mag_dir198402.nc
mag_dir198403.nc
mag_dir198404.nc
mag_dir198405.nc
mag_dir198406.nc
mag_dir198407.nc
mag_dir198408.

In [15]:
## create a class to allow for easy generation of maps, files, and dataframes
class map_generator:
    def __init__(self, list_files):
        self.file_names=list_files
     
    ## finds the max windspeed of a given year. returns a nc file
    def agg_max(self,year):
        map_files=[file for file in self.file_names if file[-9:-5]==year]
        for i,file in enumerate(map_files):
            fp=xr.open_dataset(file)
            if i==0:
                max_mag=fp.max(dim=['Time'])
            else:
                max_mag_month=fp.max(dim=['Time'])
                max_mag=xr.concat([max_mag,max_mag_month],dim='Time')
        max_value=max_mag.max(dim=['Time'])['mag']
        return max_value
    
    ## finds the mean wind speed of a given year. returns an nc file
    def agg_mean(self,year):
        map_files=[file for file in self.file_names if file[-9:-5]==year]
        for i,file in enumerate(map_files):
            fp=xr.open_dataset(file)
            if i==0:
                mean_mag=fp.mean(dim=['Time'])
            else:
                mean_mag_month=fp.mean(dim=['Time'])
                mean_mag=xr.concat([mean_mag,mean_mag_month],dim='Time')
        mean_value=mean_mag.max(dim=['Time'])['mag']
        return mean_value
    
    ## converts nc to tif file. nc file must be only one time step
    def dem_to_tif(self,dem,output_file='/shared_space/ACV/tif_files/placeholder.tif'):
        nc_file='/shared_space/ACV/inter_files/nc_to_translate.nc'
        inter_file='/shared_space/ACV/inter_files/inter_file.tif'
        output_file='/shared_space/ACV/inter_files/output_file.tif'
        if os.path.exists(nc_file):
            os.remove(nc_file)
        if os.path.exists(inter_file):
            os.remove(inter_file)
        dem.to_netcdf(nc_file)
        string_1="gdal_translate -a_srs '+proj=lcc +a=6370000 +b=6370000 +lon_0=-97.899994 +lat_0=39.100006 +lat_1=30.0 +lat_2=50.0 +datum=WGS84 +units=m +no_defs' -a_ullr 1144000.0 8000.0 2084000.0 -528000.0 %s %s"
        string_2="gdalwarp -overwrite -r average -s_srs '+proj=lcc +a=6370000 +b=6370000 +lon_0=-97.899994 +lat_0=39.100006 +lat_1=30.0 +lat_2=50.0 \
        +datum=WGS84 +units=m +no_defs' -t_srs EPSG:4326 %s %s"
        os.system(string_1%(nc_file,inter_file))
        os.system(string_2%(inter_file,output_file))
        return rs.open(output_file)
   
    ## plots tif file and intersects countys on plot. Prints intersected df as well
    def plotter(self,year,max_mean,county=True):
        if max_mean=='max':
            agg_dem=self.agg_max(year)
            print_type='Max'
        elif max_mean=='mean':
            agg_dem=self.agg_mean(year)
            print_type='Mean'
        else:
            print('cannot plot max_mean')

        plot_fp=all_data.dem_to_tif(agg_dem)
        dem=plot_fp.read(1)
        extent=[plot_fp.bounds[0],plot_fp.bounds[2],plot_fp.bounds[1],plot_fp.bounds[3]]
        
        county_fp=gpd.read_file('/shared_space/ACV/county_shp/NCDOT_County_Boundaries.shp')
        county_fp=county_fp.to_crs('EPSG:4326')
        county_fp['geometry'] = county_fp['geometry'].apply(lambda geometry: shapely.ops.transform(lambda x, y: (y, x), geometry))

        #extent=[lons.min(), lons.max(), lats.min(), lats.max()]
        fig,ax=plt.subplots(figsize=(12, 10), subplot_kw={'projection': ccrs.PlateCarree()})
        c = ax.imshow(dem, transform=ccrs.PlateCarree(), cmap='viridis',origin='upper',extent=extent) 
        ax.set_title('%s %s Wind Speed'%(year,print_type))   
        if county==True:
            ax.add_geometries(county_fp.geometry, crs = ccrs.PlateCarree(),edgecolor = 'black',facecolor='none')
        plt.show()
        
        ## intersect the shapefile with the tif
        output_arrays=county_fp.apply(lambda x: mask.mask(plot_fp, [x['geometry']])[0][0,:,:],axis=1)
        output_arrays = output_arrays.apply(lambda arr: np.nan_to_num(arr, nan=-99999))

        speeds={}
        for i,county in enumerate(output_arrays):
            speeds[county_fp['CountyName'][i]]=plot_fp.read(1)[output_arrays[i]!=-99999]
    
        results_df=pd.DataFrame([])
        results_df['County']=list(speeds.keys())
        results_df['mean']=results_df.apply(lambda x: np.mean(speeds[x['County']]),axis=1)
        results_df['max']=results_df.apply(lambda x: np.max(speeds[x['County']]),axis=1)
        
        return results_df
        
cdf_list=glob.glob('/shared_space/ACV/extracted_raw/mag_dir/*')
all_data=map_generator(cdf_list)

In [19]:
# Specify the year of interest and aggregation type
year_of_interest = '2005'
agg_type = 'max'  # Choose between 'max' or 'mean'

# Generate the aggregated DEM for the specified year and type
if agg_type == 'max':
    agg_dem = all_data.agg_max(year_of_interest)
elif agg_type == 'mean':
    agg_dem = all_data.agg_mean(year_of_interest)
else:
    raise ValueError("agg_type must be either 'max' or 'mean'.")

# Define the output file path in the current directory
output_file_path = './max_wind_speed_2005.tif'

# Save the TIFF file
saved_tif = all_data.dem_to_tif(agg_dem, output_file=output_file_path)

# Confirm the saved file
print(f"TIFF file saved at: {output_file_path}")



TIFF file saved at: ./max_wind_speed_2005.tif


In [17]:
import rasterio as rs
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import geopandas as gpd
import shapely.ops

# Path to the generated TIFF file
tif_file_path = '/shared_space/ACV/tif_files/windspeed_2005_max.tif'

# Load the TIFF file using rasterio
with rs.open(tif_file_path) as src:
    dem = src.read(1)  # Read the first band of the TIFF
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]  # Get bounds

# Set up the map projection and figure
fig, ax = plt.subplots(figsize=(12, 10), subplot_kw={'projection': ccrs.PlateCarree()})

# Plot the raster data
raster_plot = ax.imshow(dem, extent=extent, transform=ccrs.PlateCarree(), cmap='viridis', origin='upper')
ax.set_title('Maximum Wind Speed - 2005')
plt.colorbar(raster_plot, ax=ax, orientation='vertical', label='Wind Speed (units)')

# Optional: Overlay county boundaries
county_shapefile = '/shared_space/ACV/county_shp/NCDOT_County_Boundaries.shp'
county_fp = gpd.read_file(county_shapefile).to_crs('EPSG:4326')  # Ensure CRS matches the TIFF
county_fp['geometry'] = county_fp['geometry'].apply(
    lambda geometry: shapely.ops.transform(lambda x, y: (y, x), geometry)
)
ax.add_geometries(county_fp.geometry, crs=ccrs.PlateCarree(), edgecolor='black', facecolor='none')

# Show the map
plt.show()


RasterioIOError: /shared_space/ACV/tif_files/windspeed_2005_max.tif: No such file or directory